In [86]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import akshare as ak
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows=30
# pd.options.display.max_columns=30

# 1.DATA API

In [65]:
def get_us_symbol_hist_data(symbol):
    '''
        获取单只股票的每日数据
    '''
    df = ak.stock_us_hist(symbol=symbol, period="daily", start_date="20210101", end_date="20231231", adjust="hfq")
    stock_hist = df[['日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '涨跌幅']]
    stock_hist.rename(columns={'日期': 'Data', '开盘': 'Open', '收盘': 'Close', '最高': 'High', '最低': 'Low', '成交量': 'Volume', '成交额': 'Amount', '涨跌幅':'Pct_Change', '换手率':'Turnover'},inplace=True)
    stock_hist.set_index('Data',inplace=True)
    stock_hist.index = pd.to_datetime(stock_hist.index)
    
    return  stock_hist

In [37]:
def get_us_symbol_single_column(symbol_list=['105.AAPL','105.AMZN','106.GOLD'],column_name='Close'):
    '''
        获取多只股票的每日字段数据
    '''
    df_list=[]
    for symbol in symbol_list:
        df = get_us_symbol_hist_data(symbol)
        # 选取需要的数据列
        symbol_close_data = df[[f'{column_name}']].rename(columns={f'{column_name}': symbol})
        # 将单只股票的收盘价放入列表
        df_list.append(symbol_close_data)

    # 将列表中的数据合并为二维表
    combined_df = pd.concat(df_list, axis=1, join='inner')  # 使用内连接，仅保留所有数据框中都有的日期

    # 处理缺失数据（如果需要）
    combined_df = combined_df.fillna(method='ffill').fillna(method='bfill')  # 向前填充和向后填充

    return combined_df

# 2.图表绘制

In [38]:
def plot_of_daily_returns(df):
    '''
        绘制收盘价曲线图
    '''
    plt.plot(df,label=list(df.columns))

    # 添加图例和标题
    plt.title('plot_of_daily_returns')
    plt.xlabel('Time')
    plt.ylabel('Colse')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# 收盘价曲线图
df=get_us_symbol_single_column(symbol_list=['105.AAPL','105.AMZN','106.GOLD'],column_name='Close')

# 例子
plot_of_daily_returns(df)

In [40]:
def histogram_of_daily_returns(df):
    '''
        绘制收益率直方图
    '''
    plt.hist(df, bins=50)

    # 绘制均值和方差
    plt.axvline(df.mean()          , color='red',  linestyle='dashed', linewidth=2, label='mean')
    plt.axvline(+np.sqrt(df.std()) ,color='green', linestyle='dashed', linewidth=2, label='+std')
    plt.axvline(-np.sqrt(df.std()) ,color='green', linestyle='dashed', linewidth=2, label='-std')

    # 添加图例和标题
    plt.title(f'histogram_of_daily_returns')
    plt.xlabel('Pct (%)')
    plt.ylabel('fre')
    plt.legend()
    plt.grid(True)
    plt.show()

# 例子
df=get_us_symbol_hist_data('105.AAPL')['P_Change']
histogram_of_daily_returns(df)

In [42]:
def scatter_double_symbol_daily_return(df):
    '''
        两只股票收益率相关曲线
    '''
    plt.scatter(df.iloc[:,0], df.iloc[:,1])     #X轴为第一支股票的收益率 Y轴为第二只股票收益率

    # 拟合曲线
    beta,alpha= np.polyfit(df.iloc[:,0], df.iloc[:,1], deg=1)   #斜距，斜率
    plt.plot(df.iloc[:,0],beta*df.iloc[:,0]+alpha,'--',color='red')

    # 计算相关系数
    pearson_corr=df.corr(method='pearson')      #皮尔逊相关性检验
    print(pearson_corr)

    # Customize the plot (optional)
    plt.xlabel(f'{df.columns[0]}')
    plt.ylabel(f'{df.columns[1]}')
    plt.title(f'Correlation between {df.columns[0]} and {df.columns[1]}')
    plt.grid(True)  # Show grid lines 
    plt.show()

# 例子
df=get_us_symbol_single_column(['105.AAPL','106.GOLD'],'Pct_Change')
scatter_double_symbol_daily_return(df)

# 3.指标计算

### 2.1单一组合收益类计算
总收益率:
$ total-retuen=\left(\frac{P_{n}}{P_1}\right)-1 $

时点收益率:
$ position-retuen=\left(\frac{P_{m}}{P_1}\right)-1 $

注：对数计算
$ log(\frac{t4}{t1})=\log(\frac{t4}{t3})+\log(\frac{t3}{t2})+\log\left(\frac{t2}{t1}\right) $

最大回撤:$ maxdown=\max((Di-Dj)/Di) $

注：D为某一天的净值，i为某一天，j为i后的某一天，Di为第i天的产品净值，Dj则是Di后面某一天的净值

夏普比率

$ SharpeRatio=\frac{E\left(Rp\right)-Rf}{\sigma p} $

其中E(Rp)：投资组合预期年化报酬率
Rf：年化无风险利率
σp：投资组合年化报酬率的标准差

每日无风险收益率:
采用国债或者储蓄利率，按365天计算

$ 日收益率=\sqrt[365]{1+年收益率}-1 $

In [133]:
# 数据获取
single_symbol=get_us_symbol_hist_data('106.GOLD')

In [134]:
# 每日对数收益率
single_symbol['Log_Change'] = np.log(single_symbol['Close'] / single_symbol['Close'].shift(1))

In [137]:
# 时点收益率(对数)
single_symbol['Position_Return']=np.exp(single_symbol['Log_Change'].cumsum())-1

# 时点收益率(普通)
single_symbol['Position_Return']=single_symbol['Close'].div(single_symbol['Close'].iloc[0])-1

# 最终累计收益率
最终累计收益率=(single_symbol['Close'][-1]/single_symbol['Close'][0])-1

In [143]:
# 区间最大收盘价格
single_symbol['Max_Close']=single_symbol['Close'].cummax()

# 计算每个时点的最大回撤
single_symbol['Max_Drawdown']=1-single_symbol['Close']/single_symbol['Max_Close'] 

# 取其中的最大回撤
max_drawdown=max(single_symbol['Max_Drawdown'])

In [154]:
# 根据无风险年化收益率计算每日无风险收益率
daily_free_risk_rate2=pow(1+0.02, 1/365)-1
daily_free_risk_rate=0.02/265   #简化计算

sharp_ratio=(single_symbol['Pct_Change'] - daily_free_risk_rate).mean()/single_symbol['Pct_Change'].std()

### 2.2资产组合收益类计算
同2.1


In [168]:
# 获取资产组合的收盘价数据
portfolios_close=get_us_symbol_single_column(symbol_list=['105.AAPL','105.AMZN','106.GOLD','107.SPYD'],column_name='Close')

In [169]:
# 归一化收盘价,计算每天收盘价相对于第一天收盘价的百分比
portfolios_close=portfolios_close.div(portfolios_close.iloc[0], axis=1)

# 初始化资金(权重4:3:2:1)
portfolios_close=portfolios_close.apply(lambda x: x * [4000,3000,2000,1000], axis=1)

In [173]:
# 组合资产时点收盘价
portfolios_close['Portfolios_Close']=portfolios_close.sum(axis=1)      #计算每一个交易日时点的资金

$x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}$